In [2]:
pip install pandas==1.3.4

     |████████████████████████████████| 11.5 MB 3.7 MB/s eta 0:00:01    |▊                               | 245 kB 3.7 MB/s eta 0:00:04
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [1]:
%load_ext autoreload
%autoreload 2
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'


In [2]:
# !pip install fcsparser
# !pip install pandas
# !pip install scanpy
# !pip install torch

True

In [2]:
import pandas as pd
import scanpy as sc
import os
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from torch.autograd import Variable

In [3]:
# IF this doesn't work, try reading fcs natively using python
data_dir = f'/home/razgaon/ondemand/data/sys/myjobs/projects/default/3/'
NSCLC_pickle = f'{data_dir}NSCLC_prepared_data_test.pickle'
PBMC_pickle = f'{data_dir}PBMC_prepared_data_test.pickle'
NSCLC = pd.read_pickle(NSCLC_pickle)
PBMC = pd.read_pickle(PBMC_pickle)


# Try to add an id that maps to the fcs file name


In [4]:
PBMC_df = PBMC.sample(100000)
NSCLC_df = NSCLC.sample(100000)

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
excluded_markers = ['id', 'Time', 'Eventlength', 'sample', 'batch']
NSCLC_markers = [col for col in NSCLC_df.columns if col not in excluded_markers]
NSCLC_df[NSCLC_markers] = scaler.fit_transform(NSCLC_df[NSCLC_markers])

PBMC_markers = [col for col in PBMC_df.columns if col not in excluded_markers]
PBMC_df[PBMC_markers] = scaler.fit_transform(PBMC_df[PBMC_markers])


In [38]:
df = pd.concat([PBMC_df, NSCLC_df])
df = df.fillna(0)

In [27]:

class MyDataset(Dataset):
 
    def __init__(self,df):
        x=df.values
        self.x_train=torch.tensor(x,dtype=torch.float32)

    def __len__(self):
        return len(self.x_train)
   
    def __getitem__(self,idx):
        return self.x_train[idx]


In [40]:
# Convert metals to their marker names to ensure consistency when merging dataframes


,CD4589,CD45115,Xe131Di,Cs133Di,La139Di,Ce140Di,CCR6,Nd142Di,Ce142Di,CD45RA,...,CXCR3,CD3,EOMES,CD25,HuCD45,Blimp1,CD34,Tbet,CD4,CD127
534716,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.051772,0.091566,0.034135,0.000000,0.395480,0.000000,0.000000,0.038084,0.488801,0.0
928442,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.356892,...,0.229658,0.654777,0.232007,0.000000,0.504192,0.206348,0.101685,0.578465,0.235356,0.0
470436,0.0,0.0,0.0,0.0,0.0,0.0,0.666494,0.0,0.0,0.786090,...,0.000000,0.031984,0.067008,0.169429,0.489926,0.091316,0.000000,0.038084,0.000000,0.0
99922,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.470774,...,0.148218,0.458496,0.034135,0.000000,0.446984,0.091316,0.000000,0.038084,0.383272,0.0
894827,0.0,0.0,0.0,0.0,0.0,0.0,0.148435,0.0,0.0,0.728323,...,0.190910,0.411366,0.232007,0.065044,0.389667,0.091316,0.000000,0.740497,0.262387,0.0


In [41]:
markers = NSCLC_markers + PBMC_markers
dataset = MyDataset(df[markers])
# # Data loader
train_loader = DataLoader(dataset, batch_size=1000, shuffle=True)
test_loader = DataLoader(dataset, batch_size=1000, shuffle=True)



In [9]:
class VAE(nn.Module):
    def __init__(self, x_dim, latent_dim=5, hdim=50):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.latent_dim = latent_dim
        self.fc1 = nn.Linear(x_dim , hdim)
        self.fc21 = nn.Linear(hdim, latent_dim)
        self.fc22 = nn.Linear(hdim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hdim)
        self.fc4 = nn.Linear(hdim, x_dim)
        
    def encode(self, x):
        x_ = torch.cat([x], dim=-1)
        h1 = F.relu(self.fc1(x_))
        return self.fc21(h1), self.fc22(h1)
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return self.fc4(h3)
    def reparameterize(self, mu, var):
#         if self.distance == 'standard':
        if self.training:
            std = var.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            result = eps.mul(std).add_(mu)
            return result
        else:
            return mu
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, self.x_dim))
        latent = self.reparameterize(mu, logvar)
        reconstructed = self.decode(latent)
        return reconstructed, latent

In [10]:
def compute_kernel(x, y, device):
    x_size = x.to(device).size(0)
    y_size = y.to(device).size(0)
    dim = x.size(1)
    x = x.unsqueeze(1)
    y = y.unsqueeze(0)
    tiled_x = x.expand(x_size, y_size, dim).to(device)
    tiled_y = y.expand(x_size, y_size, dim).to(device)
    kernel_input = (tiled_x - tiled_y).pow(2).mean(2)/float(dim)
    return torch.exp(-kernel_input)

def compute_mmd(x, y, device):
    x_kernel = compute_kernel(x, x, device)
    y_kernel = compute_kernel(y, y, device)
    xy_kernel = compute_kernel(x, y, device)
    mmd = x_kernel.mean() + y_kernel.mean() - 2*xy_kernel.mean()
    return mmd

def vae_loss_function(inputs, encoded, decoded, true_samples, device):
    #reconstruction loss - decoding
    nll = (decoded - inputs).pow(2).mean()
    #distance + reconstruction loss
    mmd = compute_mmd(true_samples, encoded.to(device), device)
    return(nll, mmd)

In [22]:
def train(net, learning_rate, epochs, train_loader, test_loader, optimizer = 'Adam'):
    if optimizer == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr = learning_rate)
    elif optimizer == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr = learning_rate, 
                              momentum = .95, nesterov = True)
    
    for epoch in range(epochs):
        training_loss = 0
        training_reconstruction_error = 0
        training_mmd = 0

        net.train()
        for batchnum, X in enumerate(train_loader):
            optimizer.zero_grad()
            reconstruction, mu = net(X)
            true_samples = torch.randn(mu.shape)
            reconstruction_error, mmd = vae_loss_function(inputs = X, encoded=mu, decoded=reconstruction, true_samples=true_samples , device=device)
            loss = reconstruction_error + mmd
            loss.backward()
            
            optimizer.step()

            training_reconstruction_error += reconstruction_error
            training_mmd  += mmd
            training_loss += loss

        training_reconstruction_error /= (batchnum+1)
        training_mmd  /= (batchnum+1)
        training_loss /= (batchnum+1)
        print('Training loss for epoch %i is %.8f, Reconstruction is %.8f, mmd is %.8f'%(epoch, training_loss, training_reconstruction_error, training_mmd) )
        
        #Testing loop

        testing_reconstruction_error = 0
        testing_mmd = 0

        with torch.no_grad():
            for batchnum, X in enumerate(test_loader):
                reconstruction, mu = net(X)
                true_samples = torch.randn(mu.shape)
                reconstruction_error, mmd = vae_loss_function(inputs = X, encoded=mu, decoded=reconstruction, true_samples=true_samples , device=device)

                testing_reconstruction_error += reconstruction_error
                testing_mmd += mmd
            
            testing_reconstruction_error /= (batchnum+1)
            testing_mmd /= (batchnum+1)
            print('Testing loss for epoch %i is %.8f, Reconstruction is %.8f, mmd is %.8f'%(epoch, testing_reconstruction_error + testing_mmd, testing_reconstruction_error, testing_mmd) )
        

In [42]:
net = VAE(x_dim=dataset.x_train.shape[1])

In [43]:
train(net, learning_rate = .0001, epochs = 30,
      train_loader = train_loader, test_loader = test_loader)

Training loss for epoch 0 is 0.08995236, Reconstruction is 0.08726753, mmd is 0.00268484
Testing loss for epoch 0 is 0.06324264, Reconstruction is 0.06110317, mmd is 0.00213947
Training loss for epoch 1 is 0.05268922, Reconstruction is 0.05067399, mmd is 0.00201523
Testing loss for epoch 1 is 0.04492477, Reconstruction is 0.04328489, mmd is 0.00163988
Training loss for epoch 2 is 0.04054273, Reconstruction is 0.03918091, mmd is 0.00136181
Testing loss for epoch 2 is 0.03671294, Reconstruction is 0.03562937, mmd is 0.00108358
Training loss for epoch 3 is 0.03296812, Reconstruction is 0.03182631, mmd is 0.00114180
Testing loss for epoch 3 is 0.02909335, Reconstruction is 0.02762455, mmd is 0.00146879
Training loss for epoch 4 is 0.02676261, Reconstruction is 0.02471765, mmd is 0.00204496
Testing loss for epoch 4 is 0.02507382, Reconstruction is 0.02258817, mmd is 0.00248565
Training loss for epoch 5 is 0.02386331, Reconstruction is 0.02142551, mmd is 0.00243780
Testing loss for epoch 5 i

In [44]:
torch.save(net, 'model_state')

In [45]:
# Model class must be defined somewhere
model = torch.load('model_state')
model.eval()

VAE(
  (fc1): Linear(in_features=100, out_features=50, bias=True)
  (fc21): Linear(in_features=50, out_features=5, bias=True)
  (fc22): Linear(in_features=50, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=100, bias=True)
)

In [47]:
r, latent = net(train_loader[0][1])

TypeError: 'DataLoader' object is not subscriptable